In [15]:
from pathlib import Path
from dataclasses import dataclass


@dataclass(frozen=True)
class DataTransformConfig:
    root_dir: Path
    data_dir: Path
    
  

In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from mlProject.constants import *
from mlProject.utils.common import read_yaml , create_directories
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder


In [5]:
%pwd

'c:\\Users\\S Kumar\\project\\end_to_end_project_ml\\research'

In [6]:
import os

In [7]:
os.chdir('../')

In [1]:
%pwd

'c:\\Users\\S Kumar\\project\\end_to_end_project_ml\\research'

In [50]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import requests
from pathlib import Path
from dataclasses import dataclass



class ConfigurationManager:
    def __init__(self, config_file_path= CONFIG_FILE_PATH ,params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    

    def get_data_transformation_config(self)->DataTransformConfig:
        config=self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformConfig(
            root_dir=config.root_dir,
            data_dir=config.data_dir
        )
        return data_transformation_config



In [ ]:
class DataTranformation:
    def __init__(self, config: DataTransformConfig):
        self.config = config
        create_directories([self.config.root_dir])
    def lable_data(self)->pd.DataFrame:
        data = pd.read_csv(self.config.data_dir)
        categoried_1=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']]
        clarity_mapping = {'I1':0,'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}
        color_mapping = {'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6}

        encoder1=OrdinalEncoder(categories=categoried_1)
        encoder2=OrdinalEncoder(categories=clarity_mapping)
        encoder3=OrdinalEncoder(categories=color_mapping)

        data["cut"]=encoder1.fit_transform(data[["cut"]])
        data["clarity"]=data["clarity"].map(clarity_mapping)
        data["color"]=data["color"].map(color_mapping)
        return data
    
    def transform(self,df):
        df.drop(columns=["Unnamed: 0"], inplace=True)
        df.dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        Y= df[["price"]]
        X =df.drop(columns=["price"])
        X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        X_train.to_csv(os.path.join(self.config.root_dir ,"X_train.csv"), index=False)
        X_test.to_csv(os.path.join(self.config.root_dir , "X_test.csv"), index=False) 
        Y_train.to_csv(os.path.join(self.config.root_dir , "Y_train.csv"), index=False)
        Y_test.to_csv(os.path.join(self.config.root_dir , "Y_test.csv"), index=False)
        


In [72]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTranformation(config=data_transformation_config)
data=data_transformation.lable_data()
data_transformation.transform(data)


[2025-04-25 01:16:56,610]: INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-04-25 01:16:56,613]: INFO:common:yaml file: params.yaml loaded successfully]
[2025-04-25 01:16:56,617]: INFO:common:yaml file: schema.yaml loaded successfully]
[2025-04-25 01:16:56,621]: INFO:common:created directory at: artifact]
[2025-04-25 01:16:56,623]: INFO:common:created directory at: artifact/data_transformation]
[2025-04-25 01:16:56,624]: INFO:common:created directory at: artifact/data_transformation]


In [ ]:
data=